# 交差検証法（クロスバリデーション）によるパラメータチューニング

- 教師データに対するあてはまりではなく、テストデータに対する予測精度の方が重要
- クロスバリデーションにより、テストデータでパラメータチューニング

In [1]:
import pandas as pd
import seaborn as sns

# Iris データセット
iris = sns.load_dataset('iris')

# species が setosa のデータを除去
df = iris.query('species!="setosa"')
# speciesを 0, 1 にするためにダミー変数化
df = pd.get_dummies(data=df, drop_first=True)

# データセットの確認
df.iloc[[0, 50]]

,sepal_length,sepal_width,petal_length,petal_width,species_virginica
50,7.0,3.2,4.7,1.4,0
100,6.3,3.3,6.0,2.5,1


### 交差検証法

cross_val_score() にモデルとデータセットを渡し予測精度を測定
- ```cv=10``` 10分割で実行
- ```scoring='roc_auc'``` 精度はAUC<br>
指定できる精度指標は以下を参照<br>
https://scikit-learn.org/stable/modules/model_evaluation.html

In [2]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# 説明変数と目的変数
X = df[['petal_length', 'petal_width']]
Y = df.species_virginica

# SVMモデル
model = SVC()

# 交差検証の実行
score = cross_val_score(model, X, Y, cv=10, scoring='roc_auc')
print('AUC={:.3f} (+/- {:.3f})'.format(score.mean(), score.std()))

AUC=0.984 (+/- 0.027)


### 交差検証法によるパラメータチューニング

SVMのパラメータチューニングは、kernelの種類によって、指定したいオプションが異なる。

`kernel='linear'`
- `C=[0.1, 1.0, 10.0]`

`kernel='rbf'`
- `C=[0.1, 1.0, 10.0]`
- `gamma=[0.1, 1.0, 10.0]`

`kernel='poly'`
- `C=[0.1, 1.0, 10.0]`
- `degree=[3, 6, 9]`


In [ ]:
# 異なるパラメータを設定したモデルのリストの作成
models = [
    SVC(kernel='linear', C=0.1),
    SVC(kernel='linear', C=1.0),
    SVC(kernel='linear', C=10.0),
    SVC(kernel='rbf', C=0.1, gamma=0.1),
    SVC(kernel='rbf', C=0.1, gamma=1.0),
    SVC(kernel='rbf', C=0.1, gamma=10.0),
    SVC(kernel='rbf', C=1.0, gamma=0.1),
    SVC(kernel='rbf', C=1.0, gamma=1.0),
    SVC(kernel='rbf', C=1.0, gamma=10.0),
    SVC(kernel='rbf', C=10.0, gamma=0.1),
    SVC(kernel='rbf', C=10.0, gamma=1.0),
    SVC(kernel='rbf', C=10.0, gamma=10.0),
    SVC(kernel='poly', C=0.1, degree=3),
    SVC(kernel='poly', C=0.1, degree=6),
    SVC(kernel='poly', C=0.1, degree=9),
    SVC(kernel='poly', C=1.0, degree=3),
    SVC(kernel='poly', C=1.0, degree=6),
    SVC(kernel='poly', C=1.0, degree=9),
    SVC(kernel='poly', C=10.0, degree=3),
    SVC(kernel='poly', C=10.0, degree=6),
    SVC(kernel='poly', C=10.0, degree=9)
]

for i in range(0, len(models)):
    # モデルの選択
    model = models[i]

    # 交差検証の実行
    score = cross_val_score(model, X, Y, cv=10, scoring='roc_auc')
    print('{}: AUC={:.3f} (+/- {:.3f})'.format(i, score.mean(), score.std()))

### 引数を dict 型で渡す方法

可変長引数の仕組み (kwargs) を用い、引数を dict 型で定義し渡すこともできる。

In [ ]:
# 異なるパラメータを設定したモデルのリストの作成
args_dict = [
    {'kernel': 'linear', 'C': 0.1},
    {'kernel': 'linear', 'C': 1.0},
    {'kernel': 'linear', 'C': 10.0},
    {'kernel': 'rbf', 'C': 0.1, 'gamma': 0.1},
    {'kernel': 'rbf', 'C': 0.1, 'gamma': 1.0},
    {'kernel': 'rbf', 'C': 0.1, 'gamma': 10.0},
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 0.1},
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 1.0},
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 10.0},
    {'kernel': 'rbf', 'C': 10.0, 'gamma': 0.1},
    {'kernel': 'rbf', 'C': 10.0, 'gamma': 1.0},
    {'kernel': 'rbf', 'C': 10.0, 'gamma': 10.0},
    {'kernel': 'poly', 'C': 0.1, 'degree': 3},
    {'kernel': 'poly', 'C': 0.1, 'degree': 6},
    {'kernel': 'poly', 'C': 0.1, 'degree': 9},
    {'kernel': 'poly', 'C': 1.0, 'degree': 3},
    {'kernel': 'poly', 'C': 1.0, 'degree': 6},
    {'kernel': 'poly', 'C': 1.0, 'degree': 9},
    {'kernel': 'poly', 'C': 10.0, 'degree': 3},
    {'kernel': 'poly', 'C': 10.0, 'degree': 6},
    {'kernel': 'poly', 'C': 10.0, 'degree': 9}
]

for args in args_dict:
    # モデルの選択
    model = SVC(**args)

    # 交差検証の実行
    score = cross_val_score(model, X, Y, cv=10, scoring='roc_auc')
    print('{}: AUC={:.3f} (+/- {:.3f})'.format(args, score.mean(), score.std()))